In [ ]:
import os
import subprocess
import json

In [ ]:
def compile_java(file_path, class_path, log_folder_success, log_folder_fail, output_folder):
    r_errors_count = 0
    gen_r_errors_count = 0
    try:
        successful_compile = False
        # If the output directory doesn't exist, create it
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
        
        # Compile the .java file with the provided class_path and specify the output directory for .class files
        result = subprocess.run(['javac', '-cp', class_path, '-d', output_folder, file_path], capture_output=True, text=True, timeout=10)

        if result.returncode == 0:
            print(f"Successfully compiled \"{file_path}\".")
            stderr_log = result.stderr
            successful_compile = True
        else:
            warning_val = False
            num_errors = 1
            stderr_lines = result.stderr.splitlines()
            num_errors_line = stderr_lines[-1]
            try:
                num_errors = int(num_errors_line.split()[0])  # Extract the number of error(s)
            except ValueError:
                warning_val = True

            r_errors_count = result.stderr.count("error: package R does not exist")
            gen_r_errors_count = result.stderr.count("error: package gen does not exist")
            total_r_errors_count = r_errors_count + gen_r_errors_count

            # Check if all errors are related to "package R does not exist"
            if num_errors == total_r_errors_count and warning_val == False:
                print(f"Successfully compiled, ignoring {num_errors} 'package R does not exist' errors for \"{file_path}\".")
                stderr_log = "No error (Ignored 'package R does not exist' errors)"
                successful_compile = True
            else:
                if warning_val:
                    print(f"Compiled with warning.")
                    stderr_log = "Warning present"
                    successful_compile = True
                else:
                    print(f"Failed to compile \"{file_path}\".")
                    stderr_log = result.stderr
                    successful_compile = False

        # Read the content of the Java file
        with open(file_path, 'r', errors='ignore') as java_file:
            java_code = java_file.read()

        # Prepare the JSON log
        log_data = {
            "file": file_path,
            "java_code": java_code,
            "stdout": result.stdout,
            "stderr": stderr_log,
            "return_code": result.returncode
        }
        
        # Save the logs to a file named after the Java file being compiled
        log_file_name = os.path.basename(file_path).replace('.java', '.json')
        if successful_compile:
            log_file_path = os.path.join(log_folder_success, log_file_name)
        else:
            log_file_path = os.path.join(log_folder_fail, log_file_name)
        
        with open(log_file_path, 'w') as log_file:
            json.dump(log_data, log_file, indent=4)

        return successful_compile

    except subprocess.TimeoutExpired:
        print(f"Compilation of {file_path} timed out.")
        return False

In [ ]:
def calculate_success_rate(directory, class_path, log_folder_success, log_folder_fail, output_folder):
    if not os.path.exists(log_folder_success):
        os.makedirs(log_folder_success)

    if not os.path.exists(log_folder_fail):
        os.makedirs(log_folder_fail)

    java_files = [f for f in os.listdir(directory) if f.endswith('.java')]
    if not java_files:
        print("No .java files found.")
        return 0

    total_files = len(java_files)
    successful_compilations = 0

    for java_file in java_files:
        file_path = os.path.join(directory, java_file)
        if compile_java(file_path, class_path, log_folder_success, log_folder_fail, output_folder):
            successful_compilations += 1

    success_rate = (successful_compilations / total_files) * 100
    return success_rate, successful_compilations

In [ ]:
# For Zero-shot Self-Consistency

dir_type = "zero-shot"
dir_path = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/snippets_for_test/all-{dir_type}/"
jar_path = f"/home/azmain/snr_jars/"
class_path = f".:{jar_path}/*"
log_folder_success = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/logs/{dir_type}-logs/compile_success/"
log_folder_fail = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/logs/{dir_type}-logs/compile_fail/"
compiled_folder_path = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/compiled-classes/{dir_type}-compiled/" 

rate, num_successful = calculate_success_rate(dir_path, class_path, log_folder_success, log_folder_fail, compiled_folder_path)
print(f"Compilation success rate: {rate:.2f}%")
print(f"Number of successfully compiled files: {num_successful}")

In [ ]:
# For One-shot Self-Consistency

dir_type = "one-shot"
dir_path = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/snippets_for_test/all-{dir_type}/"
jar_path = f"/home/azmain/snr_jars/"
class_path = f".:{jar_path}/*"
log_folder_success = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/logs/{dir_type}-logs/compile_success/"
log_folder_fail = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/logs/{dir_type}-logs/compile_fail/"
compiled_folder_path = f"/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/compiled-classes/{dir_type}-compiled/" 

rate, num_successful = calculate_success_rate(dir_path, class_path, log_folder_success, log_folder_fail, compiled_folder_path)
print(f"Compilation success rate: {rate:.2f}%")
print(f"Number of successfully compiled files: {num_successful}")

In [ ]:
# For Getting Error Message for StatType-SO

dir_path = "/home/azmain/GitHub Codes/so_dataset/"
jar_path = "/home/azmain/snr_jars/"
class_path = f".:{jar_path}/*"
log_folder_success = "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/so_dataset_logs/compile_success/"
log_folder_fail = "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/so_dataset_logs/compile_fail/"
compiled_folder_path = "/home/azmain/GitHub Codes/Type_Inference_with_LLM/Java_Type_Inference/Results/so_dataset_compiled-classes/" 

rate, num_successful = calculate_success_rate(dir_path, class_path, log_folder_success, log_folder_fail, compiled_folder_path)
print(f"Compilation success rate: {rate:.2f}%")
print(f"Number of successfully compiled files: {num_successful}")